## План:
* Описать архитектуру модели
* Сделать функцию для обучения модели с сохранением метрик в W&B
* Затрекать метрики с разными моделями


TODO: 
* Еще нужно поработать над таргетом, привести его к общей длине

In [2]:
import os
from pathlib import Path

import pandas as pd


DATASET_PATH = Path("../datasets/")

ann_df = pd.read_parquet(DATASET_PATH / "annotation.parquet")
ann_df

,video_id,annotation,type
0,-esJrBWj2d8,"[0.39, 0.39, 0.39, 0.39, 0.39, 0.39, 0.39, 0.3...",tvsum
1,0tmA_C6XwfM,"[0.22999999999999998, 0.22999999999999998, 0.2...",tvsum
2,37rzWOQsNIw,"[0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.6...",tvsum
3,3eYKfiOEJNs,"[0.38, 0.38, 0.38, 0.38, 0.38, 0.38, 0.38, 0.3...",tvsum
4,4wU_LUjG5Ic,"[0.39, 0.39, 0.39, 0.39, 0.39, 0.39, 0.39, 0.3...",tvsum
...,...,...,...
20,Kids_Playing_In_Leaves,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",summe
21,Jumps,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",summe
22,Playing_On_Water_Slide,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",summe
23,Bike_Polo,"[0.2, 0.2, 0.2, 0.2, 0.06666666666666667, 0.06...",summe


In [3]:
ann_df['path'] = ann_df['video_id'].apply(lambda x: str(DATASET_PATH / "videos" / f"{x}.mp4"))
ann_df = ann_df.rename(columns={"label": "type"})
ann_df

,video_id,annotation,type,path
0,-esJrBWj2d8,"[0.39, 0.39, 0.39, 0.39, 0.39, 0.39, 0.39, 0.3...",tvsum,../datasets/videos/-esJrBWj2d8.mp4
1,0tmA_C6XwfM,"[0.22999999999999998, 0.22999999999999998, 0.2...",tvsum,../datasets/videos/0tmA_C6XwfM.mp4
2,37rzWOQsNIw,"[0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.6...",tvsum,../datasets/videos/37rzWOQsNIw.mp4
3,3eYKfiOEJNs,"[0.38, 0.38, 0.38, 0.38, 0.38, 0.38, 0.38, 0.3...",tvsum,../datasets/videos/3eYKfiOEJNs.mp4
4,4wU_LUjG5Ic,"[0.39, 0.39, 0.39, 0.39, 0.39, 0.39, 0.39, 0.3...",tvsum,../datasets/videos/4wU_LUjG5Ic.mp4
...,...,...,...,...
20,Kids_Playing_In_Leaves,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",summe,../datasets/videos/Kids_Playing_In_Leaves.mp4
21,Jumps,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",summe,../datasets/videos/Jumps.mp4
22,Playing_On_Water_Slide,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",summe,../datasets/videos/Playing_On_Water_Slide.mp4
23,Bike_Polo,"[0.2, 0.2, 0.2, 0.2, 0.06666666666666667, 0.06...",summe,../datasets/videos/Bike_Polo.mp4


## DataLoader

In [7]:
import torch
import torchvision
from video_summarization.video.datasets import VideoDataset
from video_summarization.video.transforms import VideoFilePathToTensor, VideoRandomCrop, VideoResize


# Requires a lot of memory
FPS = 1
FRAME_SIZE = FPS * 300 

dataset = VideoDataset(
    dataframe=ann_df,
    transform=torchvision.transforms.Compose([
        VideoFilePathToTensor(fps=FPS, padding_mode='last', max_len=FPS * 60),
        VideoResize([256, 256]),
    ])
)

data_loader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True)


# Check dataloader 
for i, videos in enumerate(data_loader):
    print(videos.size())
    if i == 4:
        break
        
        
train_dl = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True)

torch.Size([2, 3, 60, 256, 256])
torch.Size([2, 3, 60, 256, 256])
torch.Size([2, 3, 60, 256, 256])
torch.Size([2, 3, 60, 256, 256])
torch.Size([2, 3, 60, 256, 256])
torch.Size([2, 3, 60, 256, 256])


KeyboardInterrupt: 

## Model